# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Loan Data once again - this time the strutured data available through the CSV!

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/complaints.csv",
    metadata_columns=[
      "Date received", 
      "Product", 
      "Sub-product", 
      "Issue", 
      "Sub-issue", 
      "Consumer complaint narrative", 
      "Company public response", 
      "Company", 
      "State", 
      "ZIP code", 
      "Tags", 
      "Consumer consent provided?", 
      "Submitted via", 
      "Date sent to company", 
      "Company response to consumer", 
      "Timely response?", 
      "Consumer disputed?", 
      "Complaint ID"
    ]
)

loan_complaint_data = loader.load()

for doc in loan_complaint_data:
    doc.page_content = doc.metadata["Consumer complaint narrative"]

Let's look at an example document to see if everything worked as expected!

In [4]:
loan_complaint_data[0]

Document(metadata={'source': './data/complaints.csv', 'row': 0, 'Date received': '03/27/25', 'Product': 'Student loan', 'Sub-product': 'Federal student loan servicing', 'Issue': 'Dealing with your lender or servicer', 'Sub-issue': 'Trouble with how payments are being handled', 'Consumer complaint narrative': "The federal student loan COVID-19 forbearance program ended in XX/XX/XXXX. However, payments were not re-amortized on my federal student loans currently serviced by Nelnet until very recently. The new payment amount that is effective starting with the XX/XX/XXXX payment will nearly double my payment from {$180.00} per month to {$360.00} per month. I'm fortunate that my current financial position allows me to be able to handle the increased payment amount, but I am sure there are likely many borrowers who are not in the same position. The re-amortization should have occurred once the forbearance ended to reduce the impact to borrowers.", 'Company public response': 'None', 'Company'

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "LoanComplaints".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [5]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    loan_complaint_data,
    embeddings,
    location=":memory:",
    collection_name="LoanComplaints"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [6]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [9]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [10]:
naive_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided data, the most common issues with loans appear to involve problems with loan servicing, such as errors in loan balances, misapplied payments, wrongful denials of payment plans, and issues related to loan status and reporting. Many complaints also highlight difficulties in dealing with lenders or servicers, including lack of communication, incorrect information, and handling of payment options like forbearance or deferment.\n\nIn summary, the most common issue seems to be **problems with loan servicing**, including administrative errors, miscommunication, and mishandling of payments and account information.'

In [11]:
naive_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Yes, some complaints did not get handled in a timely manner. Specifically, at least one complaint, submitted to MOHELA on 03/28/25 regarding a delayed application process, was marked as "Not timely," indicating a delay in handling. Additionally, multiple complaints related to ongoing issues such as unresponsive customer service, unresolved account errors, and delays in dispute responses suggest that there have been instances where complaints were not addressed promptly.'

In [12]:
naive_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for several reasons, including:\n\n1. **Accumulation of interest during deferment or forbearance:** Borrowers often found that interest continued to grow even when payments were postponed, making repayment more difficult over time.\n\n2. **Lack of clear or timely communication from lenders or servicers:** Many borrowers were unaware of when their repayment obligations resumed or had difficulty understanding their loan status due to poor or inconsistent information.\n\n3. **Financial hardship and inability to afford payments:** Borrowers reported that increasing monthly payments would conflict with their basic living expenses, such as bills, food, and transportation.\n\n4. **Mismanagement and administrative issues:** Cases of improper loan transfers, incorrect reporting, and lack of access to accounts or understanding of balances contributed to repayment difficulties.\n\n5. **Systemic issues and lack of support:** Many borrowers felt misled about t

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [13]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(loan_complaint_data, )

We'll construct the same chain - only changing the retriever.

In [14]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [15]:
bm25_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

"Based on the provided context, the most common issue with loans appears to be related to dealing with lenders or servicers, particularly issues with the accuracy or transparency of information, handling of payments, and problematic practices like applying payments in a way that prolongs debt or charging unexpected fees. Specific recurring issues include:\n\n- Disputes over fees charged or incorrect fees\n- Difficulties in obtaining accurate loan or payment information\n- Problems with how payments are applied (e.g., not applying to principal)\n- Inaccurate or bad information about loans\n- Issues stemming from loan servicers' practices that seem predatory or untrustworthy\n\nWhile the context focuses on complaints about federal student loan servicers, these issues reflect broader common challenges in the loan process.\n\nIf you need a specific concise answer:  \nThe most common issue with loans is difficulties and disputes related to loan servicing, including miscommunication, imprope

In [16]:
bm25_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided information, all the complaints listed were responded to in a timely manner, as indicated by the "Timely response?" field being marked "Yes" for each case. Therefore, there is no evidence that any complaints were not handled in a timely manner.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for various reasons, including issues with the handling of their payment plans, lack of communication from the servicers, and problems with their payments being reversed or not properly processed. Some specific causes include:\n\n- Being steered into the wrong types of forbearance or having their loan transferred without proper notification.\n- Automatic payments being discontinued or not set up correctly, leading to missed or late payments.\n- Lack of response or help from loan servicers when borrowers seek assistance or file for deferments or forbearances.\n- Poor communication and notification from loan servicers regarding payment status, due dates, or account changes, sometimes leading to negative impacts on credit scores.\n- Servicers submitting accounts as past due without proper notice, even though borrowers made regular payments.\n\nOverall, these issues point to failures on the part of loan servicers to effectively communicate with borrow

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">
### Answer:

- Example Query: Find complaints about JuanHand online lending app payment processing errors.
- BM25 excels at finding documents containing the exact term. Embeddings might return semantically similar  results about other loan services like Billease, but when you need information about JuanHand specifically, exact matching is crucial.

</div>

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [18]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [19]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [20]:
contextual_compression_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided complaints, a common issue with loans, particularly student loans, appears to be problems related to dealing with lenders or servicers, including errors in loan balances, misapplied payments, wrongful denials of payment plans, and mishandling of account information. Many complaints also involve difficulties with understanding repayment options, accumulating interest, and issues with loan disclosures or privacy violations. \n\nIn summary, a most common issue is mismanagement or mishandling by loan servicers, leading to errors, confusion, and financial hardship for borrowers.'

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided information, there are indications that some complaints did not get handled in a timely manner. For example:\n\n- One complaint has been open since around 18 months without resolution, involving issues like account reviews and violations.\n- Another complaint involving legal violations has not received follow-up from the company.\n- A customer reported that issues with their auto-pay setup persisted for over 2-3 weeks despite multiple calls.\n\nWhile the company responses in these cases state "Yes" for timely response, the ongoing unresolved issues and extended delays suggest that some complaints were indeed not addressed in a timely manner.'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

"People often fail to pay back their loans due to a combination of factors, including lack of clear information about repayment obligations, difficulties with managing interest accumulation, and limited options for affordable repayment plans. In some cases, borrowers are unaware that they need to repay their loans or are not properly notified about changes in loan servicing or ownership, leading to missed payments or confusion about their balances. Additionally, high interest rates, compounded over time, can make it challenging to reduce the principal, especially if the borrower can only afford minimal payments, causing the debt to grow or remain unpaid. Financial hardships, lack of guidance from loan servicers, and limited access to affordable payment options like income-driven plans can ultimately contribute to borrowers' inability to fully repay their loans."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [23]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [24]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [25]:
multi_query_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided context, the most common issues with loans include:\n\n- Trouble with how payments are being handled, such as difficulty applying extra funds to principal or_payments being misapplied (e.g., primarily to interest).\n- Problems with loan servicing, including errors in balances, misapplied payments, and wrongful denials of payment plans.\n- Issues with loan balances increasing unexpectedly due to mismanagement or improper handling of interest (e.g., forbearance steering leading to interest capitalization).\n- Inaccurate or misleading credit reporting related to loan status.\n- Lack of proper communication from servicers about loan status, default notices, or changes in repayment terms.\n- Problems with accessing and verifying loan information, including identity or fraud concerns.\n- Servicer misconduct such as neglecting borrower rights, mishandling documents, or failing to process requests for deferment, consolidation, or forgiveness.\n\nOverall, the most recurri

In [26]:
multi_query_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, it appears that many complaints were handled in a timely manner, with responses marked as "Yes" for timely response, and some complaints explicitly stating responses occurred within required timeframes. \n\nHowever, there are also several complaints where responses were delayed or not received within the expected time, including cases marked as "No" for timely response, where complainants experienced wait times exceeding 15 days or where no response was received for over a year.\n\nTherefore, the answer is: **Yes, some complaints did not get handled in a timely manner.**'

In [27]:
multi_query_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

"People failed to pay back their loans for various reasons, including:\n\n1. Lack of clear information about the true cost of loans and interest accrual, leading borrowers to underestimate how much they owed.\n2. Difficulty managing interest that continued to accumulate during forbearance or deferment, making loans harder to pay off over time.\n3. Limited or confusing repayment options, with some borrowers not qualifying for loan forgiveness programs and feeling misled about available assistance.\n4. Financial hardships such as unemployment, medical issues, or homelessness, which made payments impossible.\n5. Challenges with loan servicers providing inaccurate or misleading information, poor communication, or mishandling accounts, which hindered borrowers' ability to manage or understand their loans.\n6. Errors or discrepancies in loan reporting and account status that negatively impacted credit scores and created additional obstacles to repayment.\n7. Experiences of being pushed into 

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

- A single user query might miss relevant documents due to vocabulary mismatch. For example, if a user asks "payment problems", the LLM might generate variations like billing issues, transaction errors, payment processing difficulties, or account charge problems. By combining results from all query variations, you get the union of relevant documents rather than being limited to what a single query formulation can retrieve. This significantly improves recall at the potential cost of some precision.

</div>

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [28]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = loan_complaint_data
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [29]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [30]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [31]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [32]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [33]:
parent_document_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'The most common issue with loans, based on the provided complaints, appears to be problems related to mismanagement and lack of transparency. Specific issues include:\n\n- Struggling to repay loans due to financial hardship and inadequate information about loan terms or consequences.\n- Receiving bad or incomplete information about loans from lenders or servicers.\n- Unauthorized or unexpected changes to loan terms, such as interest rate increases, incorrect balances, or lack of disclosure during processes like loan consolidation.\n- Systemic breakdowns impacting credit reporting and loan servicing, leading to errors and negative credit score impacts.\n\nIn summary, the most common issue involves borrowers facing difficulties because of insufficient or misleading information, mismanagement, or systemic errors in loan handling and reporting.'

In [34]:
parent_document_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, yes, several complaints indicate that issues were not handled in a timely manner. For example:\n\n- One complaint from 03/28/25 mentions that the consumer was told their application would be expedited and they would be contacted within 15 days, but as of the date of the complaint, no one had reached out.\n- Multiple complaints from 04/11/25 describe long wait times (up to 7 hours or more) to speak with representatives and mention that issues remained unresolved or unaddressed for extended periods.\n- Also, a complaint from 04/24/25 highlights repeated failed attempts to correct errors, with no resolution and ongoing difficulties, indicating delays and unresponsiveness.\n\nFurthermore, the metadata shows that for some complaints the response was marked "No" for timely response, which suggests that some issues did not get handled promptly.\n\nTherefore, yes, some complaints were not handled in a timely manner.'

In [35]:
parent_document_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans primarily due to financial hardship caused by inadequate post-graduation employment prospects, misrepresentations about the value of their education, and lack of transparency from the educational institutions and loan servicers. For example, one complainant was unable to secure employment in their field after attending a college that closed and did not provide real career support, leading to severe financial difficulties and difficulty in making loan payments. Others experienced issues such as being required to start payments before the grace period ended, or having their payment obligations miscommunicated or improperly managed by loan servicers. Additionally, some faced complications like unverified or disputed debt reporting, which further hindered their ability to manage or repay their loans. Overall, these issues highlight a combination of systemic mismanagement, lack of clear information, and personal financial hardships as reasons why indiv

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [36]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [37]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [38]:
ensemble_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'The most common issues with loans, based on the complaints data, include:\n\n- Dealing with lender or servicer issues such as mismanagement, incorrect reporting, or poor communication.\n- Problems with how payments are being handled, including inability to allocate payments properly or being steered into high-interest forbearances.\n- Errors and inaccuracies in loan balances, interest charges, or credit reporting.\n- Lack of transparency or proper documentation, including missing or improperly managed loan records.\n- Problems related to loan forgiveness, discharge, or discharge eligibility.\n- Unauthorized transfer of loans and improper handling of loan data.\n- Poor customer service and failure to resolve disputes effectively.\n- Issues with interest accrual and improper interest charges.\n- Misleading or incomplete information about loan terms and repayment options.\n\nIn summary, the most common and recurring issue appears to be the mishandling of loan servicing, including mismana

In [39]:
ensemble_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, yes, some complaints indicate that issues were not handled in a timely manner. For example:\n\n- Complaint ID 12973003 (from EdFinancial Services, NJ): The customer mentioned that it is over 2-3 weeks and they are still experiencing the same issue, which suggests a delay beyond the promised timeframe.\n- Complaint ID 12935889 (from MOHELA, CO): The complainant was dissatisfied with the response time, as their complaint was not resolved promptly.\n- Complaint ID 12410063 (from EdFinancial Services, CA): The customer reported ongoing struggles over nearly two and a half years, with repeated follow-ups and delays in resolution.\n\nAdditionally, some complaints explicitly mention delays or failure to resolve issues promptly, indicating that not all complaints were handled in a timely manner.\n\nIf you need a specific definitive answer: yes, some complaints did not get handled in a timely manner.'

In [40]:
ensemble_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for several reasons, including:\n\n- Lack of clear communication from loan servicers about when payments were expected to resume or if the loan had been transferred to a new servicer.\n- Difficulty in understanding or managing complex repayment options, interest accumulation, and the impact of deferment or forbearance.\n- Being misled by servicers into forbearance or consolidation without proper explanation of potential consequences like interest capitalization or loss of forgiveness eligibility.\n- Changes in loan transfer status without notification, leading to unawareness of repayment obligations.\n- Financial hardships that made monthly payments unaffordable, combined with insufficient support or guidance from lenders.\n- Errors or inaccuracies in loan balance reporting, debt management, or credit reporting, causing confusion and financial hardship.\n- Technical issues, such as trouble with online portals, misapplied payments, or inaccurate ac

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [41]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [42]:
semantic_documents = semantic_chunker.split_documents(loan_complaint_data[:20])

Let's create a new vector store.

In [43]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="Loan_Complaint_Data_Semantic_Chunks"
)

We'll use naive retrieval for this example.

In [44]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [45]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [46]:
semantic_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided complaints data, the most common issue with loans appears to be problems related to loan servicing and communication issues. Specific recurring issues include:\n\n- Trouble with how payments are being handled, such as incorrect payment amounts or re-amortization delays.\n- Difficulties in obtaining clear or accurate information about loan status, balances, or payment plans.\n- Problems with reporting and account status errors, such as loans being reported in default or delinquency falsely.\n- Issues with auto-debit setup and verification.\n- Concerns over improper reporting, breach of privacy, or illegal collection practices.\n\nOverall, challenges related to loan management, including lack of transparency, miscommunication, and administrative errors, seem to be the most prevalent issues with loans in this dataset.\n\nIf you need a specific summary or further details, please let me know!'

In [47]:
semantic_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, it appears that some complaints did not get handled in a timely manner. Specifically, multiple complaints state that the companies responded with "Closed with explanation" and there is no indication that they addressed the issues within the expected timeframe. \n\nFor example:\n- The complaint about Nelnet regarding unresponded letters and alleged misconduct indicates no subsequent response to the consumer\'s inquiries.\n- Several complaints mention that, despite the companies\' responses, the issues remain unresolved, and the consumers continue to face problems such as incorrect billing, unauthorized reporting, or lack of communication.\n\nWhile some responses are marked "Yes" for timely response, the lack of resolution or ongoing disputes suggests that not all complaints were effectively handled in a timely manner. \n\nTherefore, yes, some complaints did not get handled in a timely manner.'

In [48]:
semantic_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for various reasons, including issues with loan servicing, miscommunication, or administrative problems. Some specific reasons highlighted in the complaints include:\n\n- Receiving bad or incomplete information about their loan status or repayment terms.\n- Administrative delays or errors, such as failure to re-amortize payments after forbearance or missing payment records.\n- Disputes over the legitimacy of the debt or treatment of their loan accounts.\n- Problems with loan documentation or certification, leading to delays or complications in forgiveness or discharge.\n- Allegations of illegal reporting or collection practices, which may contribute to confusion or default status errors.\n- Personal financial difficulties, while not explicitly detailed, can also be implied as a general reason for default if borrowers are unable to meet payments.\n\nIn summary, failure to pay back loans was often related to administrative errors, communication issu

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:
Short and repetitive sentences like FAQ entries will have very similar embeddings, resulting in minimal semantic distances between consecutive sentences. FAQ structures depend on maintaining clear Q&A pairs, but semantic chunking might merge multiple Q&A pairs if they're topically similar.

The adjustments to make for the algorithm is to lower the threshold by using a more sensitive breakpoint to detect subtle semantic shifts between different FAQ topics. Additionally, implement custom logic to respect FAQ formatting patterns before applying semantic chunking. Lastly, combine semantic chunking with rule-based splitting that preserves Q&A pair integrity.

</div>

# 🤝 Breakout Room Part #2

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against each other. 
You can use the loans or bills dataset.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

</div>

##### HINTS:

- LangSmith provides detailed information about latency and cost.

# Synthetic Data Generation

### 1. NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [36]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Andrei\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### 2. Dependencies and API Keys

In [37]:
import os
import time
import getpass
import pandas as pd
from typing import List, Dict, Any

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API Key: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API Key: ")

# Enable LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "AI/LLM Engineering"

### 3. Load Dataset

In [38]:
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader

path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

In [39]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
rag_documents = text_splitter.split_documents(docs)

### 4. Data Generation

In [40]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.testset.graph import KnowledgeGraph, Node, NodeType
from ragas.testset.transforms import default_transforms, apply_transforms
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

kg = KnowledgeGraph()
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

transformer_llm = generator_llm
embedding_model = generator_embeddings
default_transforms_list = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms_list)

kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset_df = testset.to_pandas()

Applying SummaryExtractor:   0%|          | 0/20 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/60 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

### 5. Vectorstore Set-up

In [41]:
from langchain_community.vectorstores import Qdrant

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    rag_documents,
    embeddings,
    location=":memory:",
    collection_name="MainVectorStore"
)

### 6. Retrievers Set-up

##### Naive Retriever

In [42]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

##### BM25 Retriever

In [43]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(rag_documents, )

##### Contextual Compression

In [44]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

##### Multi-query Retriever

In [45]:
from langchain.retrievers.multi_query import MultiQueryRetriever

chat_model = ChatOpenAI(model="gpt-4.1-nano")
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

##### Parent Document Retriever

In [48]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models
from langchain_qdrant import QdrantVectorStore

child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="parent_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="parent_documents", 
    embedding=embeddings, 
    client=client
)

store = InMemoryStore()
parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

parent_document_retriever.add_documents(parent_docs, ids=None)

##### Ensemble Retriever

In [49]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

### 7. RAG Chain Setup

In [50]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

def create_rag_chain(retriever):
    return (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
    )

rag_chains = {
    "naive": create_rag_chain(naive_retriever),
    "bm25": create_rag_chain(bm25_retriever),
    "reranking": create_rag_chain(compression_retriever),
    "multi_query": create_rag_chain(multi_query_retriever),
    "parent_document": create_rag_chain(parent_document_retriever),
    "ensemble": create_rag_chain(ensemble_retriever)
}

### 8. Ragas Evaluation Setup

In [51]:
from ragas.evaluation import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    faithfulness,
    answer_relevancy
)
from datasets import Dataset

In [56]:
def evaluate_rag_chains(rag_chains, testset_df):
    """
    Evaluate each RAG chain using RAGAS metrics
    """
    results = []
    
    for chain_name, rag_chain in rag_chains.items():
        print(f"\n🔄 Evaluating {chain_name}...")
        
        # Create dataset for this specific chain
        chain_data = []
        
        for _, row in testset_df.iterrows():
            question = row['user_input']
            ground_truth = row['reference']
            
            try:
                # Get response from RAG chain
                response = rag_chain.invoke({"question": question})
                
                # Extract answer and contexts
                answer = response["response"].content
                contexts = [doc.page_content for doc in response["context"]]
                
                chain_data.append({
                    'question': question,
                    'answer': answer,
                    'contexts': contexts,
                    'ground_truth': ground_truth
                })
                
            except Exception as e:
                print(f"Error processing question with {chain_name}: {e}")
                continue
        
        if not chain_data:
            print(f"No valid data for {chain_name}")
            continue
            
        # Convert to Dataset format for RAGAS
        dataset = Dataset.from_list(chain_data)
        
        # Evaluate using RAGAS
        try:
            evaluation_result = evaluate(
                dataset=dataset,
                metrics=[context_precision, context_recall, faithfulness, answer_relevancy],
                llm=generator_llm,
                embeddings=generator_embeddings
            )

            # Helper function to safely extract and round metrics
            def safe_round(value, decimals=4):
                if isinstance(value, (list, tuple)):
                    # If it's a list, take the mean
                    return round(sum(value) / len(value), decimals) if value else 0.0
                elif isinstance(value, (int, float)):
                    return round(value, decimals)
                else:
                    return 0.0
            
            results.append({
                "Retriever": chain_name,
                "Context Precision": safe_round(evaluation_result["context_precision"]),
                "Context Recall": safe_round(evaluation_result["context_recall"]),
                "Faithfulness": safe_round(evaluation_result["faithfulness"]),
                "Answer Relevancy": safe_round(evaluation_result["answer_relevancy"])
            })
            
            print(f"✅ {chain_name} evaluation complete!")
            
        except Exception as e:
            print(f"Evaluation failed for {chain_name}: {e}")
            results.append({
                "Retriever": chain_name,
                "Context Precision": "Error",
                "Context Recall": "Error", 
                "Faithfulness": "Error",
                "Answer Relevancy": "Error"
            })
    
    return pd.DataFrame(results)

In [57]:
def analyze_latency_and_cost(rag_chains, testset_df, num_samples=5):
    """
    Analyze latency and cost for each RAG chain
    """
    performance_results = []
    
    # Sample questions for latency testing
    sample_questions = testset_df['user_input'].head(num_samples).tolist()
    
    for chain_name, rag_chain in rag_chains.items():
        print(f"Testing latency for {chain_name}...")
        
        total_time = 0
        successful_runs = 0
        
        for question in sample_questions:
            try:
                start_time = time.time()
                response = rag_chain.invoke({"question": question})
                end_time = time.time()
                
                total_time += (end_time - start_time)
                successful_runs += 1
                
            except Exception as e:
                print(f"Error in {chain_name}: {e}")
                continue
        
        avg_latency = total_time / successful_runs if successful_runs > 0 else float('inf')
        
        # Cost estimation based on retriever complexity
        cost_estimates = {
            "naive": "Low",
            "bm25": "Very Low", 
            "reranking": "High",  # Uses Cohere reranking API
            "multi_query": "Medium-High",  # Multiple LLM calls for query generation
            "parent_document": "Low-Medium",
            "ensemble": "Very High",  # Combines all retrievers
            "semantic": "Low-Medium"
        }
        
        performance_results.append({
            "Retriever": chain_name,
            "Average Latency (s)": round(avg_latency, 3),
            "Success Rate": f"{successful_runs}/{num_samples}",
            "Estimated Cost": cost_estimates.get(chain_name, "Unknown")
        })
    
    return pd.DataFrame(performance_results)

In [58]:
ragas_results = evaluate_rag_chains(rag_chains, testset_df)
performance_results = analyze_latency_and_cost(rag_chains, testset_df)

final_analysis = ragas_results.merge(
    performance_results[["Retriever", "Average Latency (s)", "Estimated Cost"]], 
    on="Retriever", 
    how="left"
)


🔄 Evaluating naive...


Evaluating:   0%|          | 0/44 [00:00<?, ?it/s]

✅ naive evaluation complete!

🔄 Evaluating bm25...


Evaluating:   0%|          | 0/44 [00:00<?, ?it/s]

✅ bm25 evaluation complete!

🔄 Evaluating reranking...
Error processing question with reranking: status_code: 429, body: data=None id='8e72c1a4-0909-4193-8c1f-587c81d4e273' message="You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"


...

✅ multi_query evaluation complete!

🔄 Evaluating parent_document...


Evaluating:   0%|          | 0/44 [00:00<?, ?it/s]

✅ parent_document evaluation complete!

🔄 Evaluating ensemble...


Evaluating:   0%|          | 0/44 [00:00<?, ?it/s]

✅ ensemble evaluation complete!
Testing latency for naive...
Testing latency for bm25...
Testing latency for reranking...
Testing latency for multi_query...
Testing latency for parent_document...
Testing latency for ensemble...


### Results Display and Analysis

In [59]:
print("\n" + "="*100)
print("🎯 COMPREHENSIVE RETRIEVER EVALUATION RESULTS")
print("="*100)

print("\n📊 RAGAS Metrics Results:")
print(ragas_results.to_string(index=False))

print("\n⚡ Performance Analysis:")
print(performance_results.to_string(index=False))

print("\n🔍 Combined Analysis:")
print(final_analysis.to_string(index=False))


🎯 COMPREHENSIVE RETRIEVER EVALUATION RESULTS

📊 RAGAS Metrics Results:
      Retriever  Context Precision  Context Recall  Faithfulness  Answer Relevancy
          naive             0.3964          0.8182        0.8840            0.8521
           bm25             0.3182          0.6667        0.7740            0.6970
      reranking             0.2333          0.4833        0.8981            0.7667
    multi_query             0.3449          0.8333        0.8452            0.8617
parent_document             0.3763          0.4848        0.7138            0.6105
       ensemble             0.4721          0.9091        0.9318            0.9499

⚡ Performance Analysis:
      Retriever  Average Latency (s) Success Rate Estimated Cost
          naive                2.834          5/5            Low
           bm25                1.664          5/5       Very Low
      reranking                3.154          5/5           High
    multi_query                5.408          5/5    Medium-Hi

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Analysis & Observations:

</div>

In [ ]:
def analyze_results(results_df):
    """
    Provide detailed analysis and recommendations
    """
    print("\n" + "="*100)
    print("DETAILED ANALYSIS & RECOMMENDATIONS")
    print("="*100)
    
    # Find best performers in each category
    numeric_cols = ["Context Precision", "Context Recall", "Faithfulness", "Answer Relevancy"]
    
    print("\nBEST PERFORMERS BY METRIC:")
    for col in numeric_cols:
        if col in results_df.columns:
            numeric_values = pd.to_numeric(results_df[col], errors='coerce')
            if not numeric_values.isna().all():
                best_idx = numeric_values.idxmax()
                best_retriever = results_df.loc[best_idx, "Retriever"]
                best_score = numeric_values.max()
                print(f"  {col}: {best_retriever} ({best_score:.4f})")
    
    # Calculate overall performance scores
    print(f"\nOVERALL PERFORMANCE RANKING:")
    print(f"{'Rank':<5} {'Retriever':<15} {'Avg Score':<12} {'Cost':<15} {'Latency':<12}")
    print("-" * 70)
    
    performance_scores = []
    for _, row in results_df.iterrows():
        scores = []
        for col in numeric_cols:
            if col in row:
                score = pd.to_numeric(row[col], errors='coerce')
                if not pd.isna(score):
                    scores.append(score)
        
        avg_score = sum(scores) / len(scores) if scores else 0
        performance_scores.append((row['Retriever'], avg_score, row.get('Estimated Cost', 'N/A'), row.get('Average Latency (s)', 'N/A')))
    
    # Sort by average score
    performance_scores.sort(key=lambda x: x[1], reverse=True)
    
    for i, (retriever, avg_score, cost, latency) in enumerate(performance_scores, 1):
        print(f"{i:<5} {retriever:<15} {avg_score:<12.4f} {cost:<15} {latency:<12}")
    
    # Recommendations based on use case
    print(f"\nRECOMMENDATIONS BY USE CASE:")
    
    print(f"\n1. HIGHEST ACCURACY (regardless of cost):")
    best_accuracy = performance_scores[0][0]
    print(f"   → Use '{best_accuracy}' for maximum performance")
    
    print(f"\n2. COST-CONSCIOUS (good performance, low cost):")
    cost_efficient = [x for x in performance_scores if x[2] in ['Low', 'Very Low']]
    if cost_efficient:
        print(f"   → Use '{cost_efficient[0][0]}' for balanced cost/performance")
    
    print(f"\n3. SPEED-OPTIMIZED (fastest response time):")
    latency_data = [(x[0], x[3]) for x in performance_scores if isinstance(x[3], (int, float))]
    if latency_data:
        fastest = min(latency_data, key=lambda x: x[1])
        print(f"   → Use '{fastest[0]}' for fastest responses ({fastest[1]}s)")
    
    print(f"\n4. PRODUCTION (balanced for enterprise use):")
    production_candidates = [x for x in performance_scores if x[2] in ['Low', 'Medium', 'Low-Medium'] and x[1] > 0.6]
    if production_candidates:
        print(f"   → Use '{production_candidates[0][0]}' for production deployment")
    
    print(f"\nKEY INSIGHTS:")
    print(f"• Context Precision measures how relevant retrieved chunks are")
    print(f"• Context Recall measures completeness of retrieved information")  
    print(f"• Faithfulness measures if answers are grounded in context")
    print(f"• Answer Relevancy measures if answers address the question")
    print(f"• Ensemble methods typically have highest cost but may provide best accuracy")
    print(f"• BM25 is often most cost-effective for keyword-heavy queries")
    print(f"• Reranking adds cost but can significantly improve precision")


In [61]:
analyze_results(final_analysis)


DETAILED ANALYSIS & RECOMMENDATIONS

🏆 BEST PERFORMERS BY METRIC:
  Context Precision: ensemble (0.4721)
  Context Recall: ensemble (0.9091)
  Faithfulness: ensemble (0.9318)
  Answer Relevancy: ensemble (0.9499)

OVERALL PERFORMANCE RANKING:
Rank  Retriever       Avg Score    Cost            Latency     
----------------------------------------------------------------------
1     ensemble        0.8157       Very High       7.538       
2     naive           0.7377       Low             2.834       
3     multi_query     0.7213       Medium-High     5.408       
4     bm25            0.6140       Very Low        1.664       
5     reranking       0.5954       High            3.154       
6     parent_document 0.5464       Low-Medium      1.828       

RECOMMENDATIONS BY USE CASE:

1. HIGHEST ACCURACY (regardless of cost):
   → Use 'ensemble' for maximum performance

2. COST-CONSCIOUS (good performance, low cost):
   → Use 'naive' for balanced cost/performance

3. SPEED-OPTIMIZED (fas